In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc


In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 64
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 128
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, Data]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000_000)):
    val = fake.name() + " - " + str(i)
    resp = index.insert(MyKey(val))
    if (resp[0]) is None:
        print(f"Duplicate key: {val}: {resp[1]}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 8min 51s, sys: 7.61 s, total: 8min 59s
Wall time: 8min 56s


In [10]:
%%time
index.validate(), gc.collect()

CPU times: user 7.08 s, sys: 9.78 ms, total: 7.09 s
Wall time: 7.09 s


({}, 0)

In [11]:
len(data), len(data) / (60 * 9)

(10000000, 18518.51851851852)

In [12]:
index.print_node(index.tree[index.root_node])


node-idx:13314, prev:None, next:None, level: 3  <============ *ROOT*
         |-> MyKey(full_name='Валентина Кирилловна Харитонова - 17651'): 132, 37598
         |-> MyKey(full_name='Евдокимов Ульян Витальевич - 1499106'): 37598, 24849
         |-> MyKey(full_name='Козлова Елизавета Станиславовна - 27291'): 24849, 48978
         |-> MyKey(full_name='Матвеева Фёкла Степановна - 159'): 48978, 13313
         |-> MyKey(full_name='Оксана Никифоровна Жданова - 849'): 13313, 54330
         |-> MyKey(full_name='Селиверстов Мирон Владиленович - 359'): 54330, 29050


In [13]:
index.print_node(index.tree[132])


node-idx:132, prev:None, next:37598, level: 2 
      |-> MyKey(full_name='Август Арсенович Миронов - 236371'): 1, 28629
      |-> MyKey(full_name='Авдеева Лукия Артемовна - 1424958'): 28629, 14570
      |-> MyKey(full_name='Аверьян Гертрудович Лобанов - 1464013'): 14570, 28811
      |-> MyKey(full_name='Авксентий Феофанович Кабанов - 6369923'): 28811, 57242
      |-> MyKey(full_name='Агап Гаврилович Лебедев - 271752'): 57242, 7197
      |-> MyKey(full_name='Агата Мироновна Ефимова - 67343'): 7197, 30479
      |-> MyKey(full_name='Агафонов Владлен Гурьевич - 263729'): 30479, 15376
      |-> MyKey(full_name='Агафья Аскольдовна Кулакова - 430222'): 15376, 30222
      |-> MyKey(full_name='Агафья Руслановна Хохлова - 100460'): 30222, 3877
      |-> MyKey(full_name='Адам Филимонович Давыдов - 91164'): 3877, 33121
      |-> MyKey(full_name='Аким Демидович Калашников - 456588'): 33121, 16438
      |-> MyKey(full_name='Аксенова Нонна Эльдаровна - 13223'): 16438, 31845
      |-> MyKey(full_name

In [15]:
index.print_node(index.tree[1])


node-idx:1, prev:None, next:28629, level: 1 
   |-> MyKey(full_name='Абрамов Аггей Яковлевич - 1897518'): 0, 28628
   |-> MyKey(full_name='Абрамов Ананий Евстигнеевич - 1461042'): 28628, 14384
   |-> MyKey(full_name='Абрамов Аникей Ефимьевич - 8100346'): 14384, 51470
   |-> MyKey(full_name='Абрамов Аполлинарий Виленович - 2496068'): 51470, 26675
   |-> MyKey(full_name='Абрамов Аркадий Викентьевич - 7223002'): 26675, 53776
   |-> MyKey(full_name='Абрамов Архип Викторович - 95675'): 53776, 6983
   |-> MyKey(full_name='Абрамов Бажен Зиновьевич - 5417453'): 6983, 47693
   |-> MyKey(full_name='Абрамов Боян Венедиктович - 500598'): 47693, 23693
   |-> MyKey(full_name='Абрамов Валентин Иосифович - 633854'): 23693, 50459
   |-> MyKey(full_name='Абрамов Василий Жоресович - 341890'): 50459, 12919
   |-> MyKey(full_name='Абрамов Викентий Ефремович - 8790562'): 12919, 52067
   |-> MyKey(full_name='Абрамов Владилен Виленович - 891813'): 52067, 26413
   |-> MyKey(full_name='Абрамов Влас Игнатович -

In [16]:
index.print_node(index.tree[0])


node-idx:0, prev:None, next:28628, level: 0 
|-> MyKey(full_name='Абрамов Август Антипович - 2077679')
|-> MyKey(full_name='Абрамов Август Антонович - 4619894')
|-> MyKey(full_name='Абрамов Август Антонович - 8514775')
|-> MyKey(full_name='Абрамов Август Артурович - 120982')
|-> MyKey(full_name='Абрамов Август Артёмович - 5292234')
|-> MyKey(full_name='Абрамов Август Бенедиктович - 8533027')
|-> MyKey(full_name='Абрамов Август Брониславович - 9824676')
|-> MyKey(full_name='Абрамов Август Валерианович - 9567864')
|-> MyKey(full_name='Абрамов Август Вилорович - 4393802')
|-> MyKey(full_name='Абрамов Август Витальевич - 4032494')
|-> MyKey(full_name='Абрамов Август Владиславович - 8936791')
|-> MyKey(full_name='Абрамов Август Власович - 6983371')
|-> MyKey(full_name='Абрамов Август Гавриилович - 4371322')
|-> MyKey(full_name='Абрамов Август Гаврилович - 8206870')
|-> MyKey(full_name='Абрамов Август Герасимович - 8801497')
|-> MyKey(full_name='Абрамов Август Данилович - 2536355')
|-> MyKe

In [22]:
index.min()[1]

MyKey(full_name='Абрамов Август Антипович - 2077679')

In [23]:
index.print_node(index.max()[0])


node-idx:45913, prev:23794, next:None, level: 0 
|-> MyKey(full_name='тов. Якушева Елизавета Валентиновна - 3419865')
|-> MyKey(full_name='тов. Якушева Елизавета Ильинична - 2395298')
|-> MyKey(full_name='тов. Якушева Елизавета Константиновна - 8019064')
|-> MyKey(full_name='тов. Якушева Елизавета Львовна - 4670940')
|-> MyKey(full_name='тов. Якушева Елизавета Романовна - 4418601')
|-> MyKey(full_name='тов. Якушева Елизавета Рудольфовна - 752734')
|-> MyKey(full_name='тов. Якушева Елизавета Филипповна - 884719')
|-> MyKey(full_name='тов. Якушева Жанна Дмитриевна - 4832511')
|-> MyKey(full_name='тов. Якушева Жанна Евгеньевна - 4081811')
|-> MyKey(full_name='тов. Якушева Жанна Святославовна - 4674027')
|-> MyKey(full_name='тов. Якушева Зинаида Архиповна - 7528920')
|-> MyKey(full_name='тов. Якушева Зинаида Вячеславовна - 1758538')
|-> MyKey(full_name='тов. Якушева Зинаида Захаровна - 8525698')
|-> MyKey(full_name='тов. Якушева Зинаида Кирилловна - 1898240')
|-> MyKey(full_name='тов. Яку

In [24]:
index.max()[1]

MyKey(full_name='тов. Якушева Юлия Павловна - 3134824')